<a href="https://colab.research.google.com/github/DeepInsider/playground-data/blob/master/docs/articles/pytorch_neuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 連載『PyTorch入門』のノートブック（1）

<table valign="middle">
  <td>
    <a target="_blank" href="https://www.atmarkit.co.jp/ait/subtop/features/di/pytorch_index.html"> <img src="https://re.deepinsider.jp/img/ml-logo/manabu.svg"/>Deep Insiderで記事を読む</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/DeepInsider/playground-data/blob/master/docs/articles/pytorch_neuralnetwork.ipynb"> <img src="https://re.deepinsider.jp/img/ml-logo/gcolab.svg" />Google Colabで実行する</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/DeepInsider/playground-data/blob/master/docs/articles/pytorch_neuralnetwork.ipynb"> <img src="https://re.deepinsider.jp/img/ml-logo/github.svg" />GitHubでソースコードを見る</a>
  </td>
</table>

# 第1回　難しくない！　PyTorchでニューラルネットワークの基本

## ■PyTorchとは？

- 人気急上昇中（参考：「[PyTorch vs. TensorFlow、ディープラーニングフレームワークはどっちを使うべきか問題 (1/2)：気になるニュース＆ネット記事 - ＠IT](https://www.atmarkit.co.jp/ait/articles/1910/31/news028.html)）」
- Pythonic（＝Pythonのイディオムをうまく活用した自然なコーディングが可能）
- 柔軟性や拡張性に優れる（特に“define-by-run”：実行しながら定義／eager execution：即時実行なので、例えばモデルのフォワード時にif条件やforループなどの制御フローを書いて動的に計算グラフを変更したりできる）

特にNLP（Natural Language Processing：自然言語処理）の分野では、研究者はさまざまな長さの文を訓練する必要性があるため、動的な計算グラフが不可欠。実際に「PyTorchがデファクトスタンダードになっている」と筆者が初めて聞いたのは、NLPに関しての話だった。

## ■本稿の目的と方針

PyTorchでニューラルネットワークを定義するための最重要の基礎知識を最短で紹介する。

- PyTorchのチュートリアルは最初からCNNで最初から複雑（※これはある程度のニューラルネットワークの知識がない人を門前払いする意味があると思う）
- まずはニューラルネットワークの原型「ニューロン」を実装することで、核となる機能を理解する
- 「ニューロン」「活性化関数」「正則化」「勾配」「確率的勾配降下法（SGD）」といった概念が分からない場合は、『[TensorFlow 2＋Keras（tf.keras）入門 - ＠IT](https://www.atmarkit.co.jp/ait/subtop/features/di/tf2keras_index.html)』の第1回～第3回で挙動を示しながら分かりやすく説明しているので、先にそちらを一読してほしい
- 最終的には、基本的なニューラルネットワーク＆ディープラーニングのコードが思いどおりに書けるようになる

## ■本稿で説明する大まかな流れ

- （1）ニューロンのモデル定義
- （2）フィードフォワード（順伝播）
- （3）バックプロパゲーション（逆伝播）と自動微分（Autograd）
- （4）PyTorchの基礎： テンソルとデータ型
- （5）データセットとデーターローダー（DataLoader）
- （6）ディープニューラルネットのモデル定義
- （7）学習／最適化（オプティマイザー）
- （8）評価／精度検証

## ■（1）ニューロンのモデル定義

###  【チェック】Pythonバージョン（※3系を使うこと）
Colabにインストール済みのものを使う。もし2系になっている場合は、メニューバーの［ランタイム］－［ランタイムのタイプを変更］をクリックして切り替えてほしい。

In [0]:
import sys
print('Python', sys.version)
# Python 3.6.9 (default, Nov  7 2019, 10:44:02)  …… などと表示される

###  【チェック】PyTorchバージョン
基本的にはColabにインストール済みのものを使う。

In [0]:
import torch
print('PyTorch', torch.__version__)
# PyTorch 1.3.1 ……などと表示される

### リスト1-0　［オプション］ライブラリ「PyTorch」最新バージョンのインストール

In [0]:
#!pip install torch        # ライブラリ「PyTorch」をインストール
#!pip install torchvision  # 画像／ビデオ処理のPyTorch用追加パッケージもインストール

# 最新バージョンにアップグレードする場合
!pip install --upgrade torch torchvision

# バージョンを明示してアップグレードする場合
#!pip install --upgrade torch==1.4.0 torchvision==0.5.0

# 最新バージョンをインストールする場合
#!pip install torch torchvision

# バージョンを明示してインストールする場合
#!pip install torch==1.4.0 torchvision==0.5.0

このコードのポイント：
- 「torchvision」パッケージは本稿では使っていないが、同時にインストールしないとパッケージ関係が不整合となるため、インストールしておく必要がある
- 実行後にランタイムを再起動する必要がある

### ［オプション］【チェック】PyTorchバージョン（※インストール後の確認）
バージョン1.4.0以上になっているか再度チェックする。

In [0]:
import torch
print('PyTorch', torch.__version__)
# PyTorch 1.4.0 ……などと表示される

### リスト1-1　ニューロンのモデル設計と活性化関数

- ニューロンへの入力＝$(w_1 \times X_1)+(w_2 \times X_2)+b$
- ニューロンからの出力＝$a((w_1 \times X_1)+(w_2 \times X_2)+b)$
  - $a()$は活性化関数を意味する。つまりニューロンの入力結果を、活性化関数で変換したうえで、出力する
  - 今回の活性化関数は、**tanh**関数とする
- ニューロンの構造とデータ入力：座標$(X_1, X_2)$
  - 入力の数（`INPUT_FEATURES`）は、$X_1$と$X_2$で**2つ**
  - ニューロンの数（`OUTPUT_NEURONS`）は、**1つ**

In [0]:
import torch       # ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn  # 「ニューラルネットワーク」モジュールの別名定義

# 定数（モデル定義時に必要となるもの）
INPUT_FEATURES = 2  # 入力（特徴）の数： 2
OUTPUT_NEURONS = 1  # ニューロンの数： 1

# 変数（モデル定義時に必要となるもの）
activation = torch.nn.Tanh()  # 活性化関数： tanh関数

# 「torch.nn.Moduleクラスのサブクラス化」によるモデルの定義
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 層（layer：レイヤー）を定義
        self.layer1 = nn.Linear(  # Linearは「全結合層」を指す
            INPUT_FEATURES,       # データ（特徴）の入力ユニット数
            OUTPUT_NEURONS)       # 出力結果への出力ユニット数

    def forward(self, input):
        # フィードフォワードを定義
        output = activation(self.layer1(input))  # 活性化関数は変数として定義
        # 「出力＝活性化関数（第n層（入力））」の形式で記述する。
        # 層（layer）を重ねる場合は、同様の記述を続ければよい（第3回＝後述）。
        # 「出力（output）」は次の層（layer）への「入力（input）」に使う。
        # 慣例では入力も出力も「x」と同じ変数名で記述する（よって以下では「x」と書く）
        return output

# モデル（NeuralNetworkクラス）のインスタンス化
model = NeuralNetwork()
model   # モデルの内容を出力

このコードのポイント：
- `torch.nn.Module`クラスを継承して独自にモデル用クラスを定義する。Pythonの「モジュール」と紛らわしいので、本稿では「`torch.nn.Module`」と表記する
    - `__init__`関数にレイヤー（層）を定義する
    - `forward`関数にフィードフォワード（＝活性化関数で変換しながらデータを流す処理）を実装する
    - ちなみにバックプロパゲーション（誤差逆伝播）のための`backward`関数は自動微分機能により自動作成される（後述）

In [0]:
#@title tanh関数
# This code will be hidden when the notebook is loaded.

import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

def tanh(x):
  return np.tanh(x)

x = np.arange(-6.0, 6.0, 0.001)
plt.plot(x, sigmoid(x), label = "Sigmoid")
plt.plot(x, tanh(x), label = "tanh")
plt.xlim(-6, 6)
plt.ylim(-1.2, 1.2)
plt.grid()
plt.legend()
plt.show()

- PyTorchでは、以下の活性化関数が用意されている
  - ELU
  - Hardshrink
  - Hardtanh
  - LeakyReLU
  - LogSigmoid
  - MultiheadAttention
  - PReLU
  - ReLU（有名）
  - ReLU6
  - RReLU
  - SELU
  - CELU
  - GELU
  - Sigmoid（シグモイド）
  - Softplus（ソフトプラス）
  - Softshrink
  - Softsign（ソフトサイン）
  - Tanh（本稿で使用）
  - Tanhshrink
  - Threshold
  - Softmin
  - Softmax
  - Softmax2d
  - LogSoftmax
  - AdaptiveLogSoftmaxWithLoss

### リスト1-2　パラメーター（重みとバイアス）の初期値設定

- $w_1=0.6$、$w_2=-0.2$、$b=0.8$と仮定して、ニューロンのモデルを定義
  - ※これらの値は通常は学習により決定されるが、今回は未学習なので仮の固定数値としている
  - 重さ（$w_1$と$w_2$）は2次元配列でまとめて表記する： `weight_array`
    - 通常は、ニューロンは複数あるので、2次元配列で表記する
    - 複数の重みが「行」を構成し、複数のニューロンが「列」を構成する
    - 今回は、重みが**2つ**で、ニューロンが**1つ**なので、**2行1列**で記述する
    -  `[[ 0.6],`<br>&nbsp;&nbsp;`[-0.2]]`
  - バイアス（$b$）は1次元配列でまとめて表記する： `bias_array`
    - `[0.8]`

In [0]:
# パラメーター（ニューロンへの入力で必要となるもの）の定義
weight_array = nn.Parameter(
    torch.tensor([[ 0.6,
                   -0.2]]))  # 重み
bias_array = nn.Parameter(
    torch.tensor([  0.8 ]))  # バイアス

# 重みとバイアスの初期値設定
model.layer1.weight = weight_array
model.layer1.bias = bias_array

# torch.nn.Module全体の状態を辞書形式で取得
params = model.state_dict()
#params = list(model.parameters()) # このように取得することも可能
params
# 出力例：
# OrderedDict([('layer1.weight', tensor([[ 0.6000, -0.2000]])),
#              ('layer1.bias', tensor([0.8000]))])

このコードのポイント：
- モデルのパラメーターは`torch.nn.Parameter`オブジェクトとして定義する
  - `torch.nn.Parameter`クラスのコンストラクター（厳密には`__init__`関数）には`torch.Tensor`オブジェクト（以下、テンソル）を指定する
  - `torch.Tensor`のコンストラクターにはPythonの多次元リストを指定できる
  - NumPyの多次元配列からのテンソルの作成や、テンソルの使い方については第2回（＝後述）
- 重みやバイアスの初期値設定：
  - `＜モデル名＞.＜レイヤー名＞.weight`プロパティに重みが指定できる
  - `＜モデル名＞.＜レイヤー名＞.baias`プロパティにバイアスが指定できる
  - 通常は「**0**」や「一様分布の**ランダムな値**」などを指定する（第3回＝後述）
- 重みやバイアスといったパラメーターなどの`torch.nn.Module`全体の状態は、`＜モデル名＞.state_dict()`メソッドで取得できる
  - ちなみにパラメーターを最適化で使う際は、`＜モデル名＞.parameters()`メソッドで取得する（第3回＝後述）

## ■（2）フィードフォワード（順伝播）

### リスト2-1　フィードフォワードの実行と結果確認
- ニューロンに、座標$(X_1, X_2)$データを入力する
  - 通常のデータは表形式（＝2次元配列）だが、今回は$(1.0, 2.0)$という1つのデータ
    - 1つのデータでも2次元配列（具体的には**1行2列**）で表現する必要がある

In [0]:
X_data = torch.tensor([[1.0, 2.0]])  # 入力する座標データ（1.0、2.0）
print(X_data)
# tensor([[1., 2.]]) ……などと表示される

y_pred = model(X_data)  # このモデルに、データを入力して、出力を得る（＝予測：predict）
print(y_pred)
# tensor([[0.7616]], grad_fn=<TanhBackward>) ……などと表示される

このコードのポイント：
- フィードフォワード（順伝播）で、データ（`X_data`）を入力し、モデル（`model`）が推論した結果（`y_pred`）を出力している
- その結果の数値は、手動で計算した値（`0.7616`）と同じになるのが確認できるはず
- `grad_fn`属性（この例では「TanhBackward」）には、勾配（偏微分）などを計算するための関数が自動作成されている。バックプロパゲーション（逆伝播）による学習の際に利用される

### リスト2-2　動的な計算グラフの可視化

「[torchviz · PyPI](https://pypi.org/project/torchviz/)」をインストールして、動的な計算グラフ（dynamic computation graph）を可視化する。


In [0]:
!pip install torchviz          # 初回の「torchviz」パッケージインストール時にのみ必要

In [0]:
from torchviz import make_dot  # 「torchviz」モジュールから「make_dot」関数をインポート
make_dot(y_pred, params=dict(model.named_parameters()))
# 引数「params」には、全パラメーターの「名前: テンソル」の辞書を指定する。
# 「dict(model.named_parameters())」はその辞書を取得している

この図のポイント：
- 青色のボックス： 勾配を計算する必要がある、重みやバイアスなどのパラメーター。この例では`(1, 2)`が重みで、`(1)`がバイアス
- 灰色のボックス： 勾配（偏微分）などを計算するための関数。「テンソル」データの`grad_fn`属性（この例では「TBackward」や「AddmmBackward」）に自動作成されている。バックプロパゲーション（逆伝播）による学習の際に利用される
- 緑色のボックス： グラフ計算の開始点。`backward()`メソッドを呼び出すと、ここから逆順に計算していく。内容は灰色のボックスと同じ

## ■（3）バックプロパゲーション（逆伝播）と自動微分（Autograd）

### リスト3-1　簡単な式で自動微分してみる

`backward()`メソッドでバックプロパゲーション（誤差逆伝播）をさせる。ニューラルネットワークの誤差逆伝播では、「微分係数（derivative）の計算」という面倒くさい処理が待っている。ディープラーニングのライブラリは、この処理を自動化してくれるので大変便利である。この機能を「自動微分（AD： Automatic differentiation）」や「Autograd」（gradients computed automatically： 自動計算された勾配）などと呼ぶ。

ちなみに詳細を知る必要はあまりないが、`torch.autograd`モジュールは厳密には「リバースモードの自動微分」機能を提供しており、vector-Jacobian product（VJP：ベクトル-ヤコビアン積）と呼ばれる計算を行うエンジンである（参考「[Autograd: Automatic Differentiation — PyTorch Tutorials 1.4.0 documentation](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py)」、論文「[Automatic differentiation in PyTorch | OpenReview](https://openreview.net/forum?id=BJJsrmfCZ)」）。

PyTorchの自動微分（Autograd）機能を、非常にシンプルな例で示しておく。

- 計算式： $y=x^2$
- 導関数： $\frac{dy}{dx}=2x$ （ $y$ を $x$ で微分する）
- 例えば $x$ が__1.0__の地点の勾配（＝接線の傾き）は__2.0__となる

In [0]:
x = torch.tensor(1.0, requires_grad=True)  # 今回は入力に勾配（gradient）を必要とする
# 「requires_grad」が「True」（デフォルト：False）の場合、
# torch.autogradが入力テンソルに関するパラメーター操作（勾配）を記録するようになる

#x.requires_grad_(True)  # 「requires_grad_()」メソッドで後から変更することも可能

y = x ** 2     # 「yイコールxの二乗」という計算式の計算グラフを構築
print(y)       # tensor(1., grad_fn=<PowBackward0>) ……などと表示される

y.backward()   # 逆伝播の処理として、上記式から微分係数（＝勾配）を計算（自動微分：Autograd）

g = x.grad     # 与えられた入力（x）によって計算された勾配の値（grad）を取得
print(g)       # tensor(2.)  ……などと表示される
# 計算式の微分係数（＝勾配）を計算するための導関数は「dy/dx=2x」なので、
#「x=1.0」地点の勾配（＝接線の傾き）は「2.0」となり、出力結果は正しい。
# 例えば「x=0.0」地点の勾配は「0.0」、「x=10.0」地点の勾配は「20.0」である

このコードのポイント：
- PyTorchが「自動微分」機能を持つライブラリであることが確認できた
- 出力されたテンソル（`y`）の`backward()`メソッドでバックプロパゲーション（逆伝播）を実行できる。なお、ニューラルネットワークの場合は、損失を表すテンソルの`backward()`メソッドを呼び出すことになる
  - 出力されたテンソルの計算式（`y`）を入力したテンソル(`x`)で微分計算している
- 計算された微分係数（＝勾配：gradient）は、入力したテンソルの`grad`プロパティで取得できる

### リスト3-2　ニューラルネットワークにおける各パラメーターの勾配

In [0]:
# 勾配計算の前に、各パラメーター（重みやバイアス）の勾配の値（grad）をリセットしておく
model.layer1.weight.grad = None      # 重み
model.layer1.bias.grad = None        # バイアス
#model.zero_grad()                   # これを呼び出しても上記と同じくリセットされる

X_data = torch.tensor([[1.0, 2.0]])  # 入力データ（※再掲）
y_pred = model(X_data)               # 出力結果（※再掲）
y_true = torch.tensor([[1.0]])       # 正解ラベル

criterion = nn.MSELoss()             # 誤差からの損失を測る「基準」＝損失関数
loss = criterion(y_pred, y_true)     # 誤差（出力結果と正解ラベルの差）から損失を取得
loss.backward()   # 逆伝播の処理として、勾配を計算（自動微分：Autograd）

# 勾配の値（grad）は、各パラメーター（重みやバイアス）から取得できる
print(model.layer1.weight.grad) # tensor([[-0.2002, -0.4005]])  ……などと表示される
print(model.layer1.bias.grad)   # tensor([-0.2002])  ……などと表示される
# ※パラメーターは「list(model.parameters())」で取得することも可能

このコードのポイント：
- `criterion`に損失関数を代入するのが定石
- `backward`メソッドによるバックプロパゲーション
- この例では単純にするために1回しか処理してないが、本来はミニバッチのイテレーションや全体のエポックの回数繰り返し処理必要がある（第3回＝後述）
- モデルにおける各パラメーター（`weight`や`bias`）の`grad`プロパティから、勾配の値は取得できる

# 第2回　PyTorchのテンソル＆データ型のチートシート

## ■（4）PyTorchの基礎： テンソルとデータ型

### 表4-1　PyTorchのデータ型

先ほどのリスト1-3では、`torch.tensor([0.8])`というコードでPyTorchのテンソル（`torch.Tensor`値）を作成した。PyTorchでデータや数値を扱うには、このテンソル形式にする必要がある。  
この例で分かるように、Pythonの数値やリスト値を`torch.Tensor`値に変換するのは難しくない。  
ここでは、テンソルを作成／変換する基本的なコードをチートシート的に書き出しておく。  
よく分からないところがあれば、下記の公式チュートリアルの説明を参照してほしい（※Chromeの［日本語に翻訳］機能を使えば、日本語で読める）。
- 公式チュートリアル： [What is PyTorch? — PyTorch Tutorials 1.4.0 documentation](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py)
- APIドキュメント： [torch — PyTorch master documentation](https://pytorch.org/docs/stable/torch.html#tensors)

また、テンソルの中に含める数値には、PyTorch独自のデータ型（`torch.dtypes`）がある。ただし統一データ型（ある1つのテンソル内に含まれる全要素の数値は全て同じデータ型）となっており、例えば`torch.float`の場合は、全ての要素の数値が「32-bitの浮動小数点」として扱われるので注意してほしい。基本的に`torch.float`か`torch.int`しか使わない。

データ型 | dtype属性への記述 | 対応するPython／NumPy（np）のデータ型
---------|-----------------|--------------------------------
Boolean（真偽値）|torch.bool|bool／np.bool
8-bitの符号なし整数|torch.uint8|int／np.uint8
8-bitの符号付き整数|torch.int8|int／np.int8
16-bitの符号付き整数|torch.int16 ／ torch.short|int／np.uint16
32-bitの符号付き整数|torch.int32 ／ torch.int|int／np.uint32
64-bitの符号付き整数|torch.int64 ／ torch.long|int／np.uint64
16-bitの浮動小数点|torch.float16 ／ torch.half|float／np.float16
32-bitの浮動小数点|torch.float32 ／ torch.float|float／np.float32
64-bitの浮動小数点|torch.float64 ／ torch.double|float／np.float64


### リスト4-1　チートシート「テンソルの新規作成とサイズ取得／変換」

以下では、シンプルにテンソルの使い方だけを示すためため、`print(x)`などの出力に関するコードは極力、省略した。コードを実行して出力を確認したい場合は、例えば`x = torch.empty(2, 3)`というコードの後に`print(x)`を追記してほしい。また、`x.size()`というコードは、`print(x.size())`のように`print`関数を適宜、自分で補ってほしい。

In [0]:
import torch
import numpy as np

# テンソルの新規作成
x = torch.empty(2, 3) # 2行×3列のテンソル（未初期化状態）を生成
x = torch.rand(2, 3)  # 2行×3列のテンソル（ランダムに初期化）を生成
x = torch.zeros(2, 3, dtype=torch.float) # 2行×3列のテンソル（0で初期化、torch.float型）を生成
x = torch.ones(2, 3, dtype=torch.float)  # 2行×3列のテンソル（1で初期化、torch.float型）を生成
x = torch.tensor([[0.0, 0.1, 0.2],
                  [1.0, 1.1, 1.2]])      # 1行×2列のテンソルをPythonリスト値から作成

# 既存のテンソルを使った新規作成
# 「new_*()」パターン
y = x.new_ones(2, 3)   # 2行×3列のテンソル（1で初期化、既存のテンソルと「同じデータ型」）を生成
# 「*_like()」パターン # 既存のテンソルと「同じサイズ」のテンソル（1で初期化、torch.int型）を生成
y = torch.ones_like(x, dtype=torch.int) 

### リスト4-2　チートシート「テンソルのサイズ取得とサイズ変更」

In [0]:
# テンソルサイズの取得
x.size()               # 「torch.Size([2, 3])」のように、2行3列と出力される
x.shape                # NumPy風の記述も可能。出力は上と同じ
len(x)   # 行数（＝データ数）を取得する際も、NumPy風に記述することが可能
x.ndim   # テンソルの次元数を取得する際も、NumPy風に記述が可能

# テンソルのサイズ変更／形状変更
z = x.view(3, 2)       # 3行2列に変更

### リスト4-3　チートシート「テンソルの演算／計算」

In [0]:
# テンソルの計算操作
x + y                  # 演算子を使う方法
torch.add(x, y)        # 関数を使う方法
torch.add(x, y, out=x) # outパラメーターで出力先の変数を指定可能
x.add_(y)              # 「*_()」パターン。xを置き換えて出力する例（上記のコードと同じ処理）
# PyTorchでは、メソッド名の最後にアンダースコア（_）がある場合（例えば「add_()」）、「テンソルの内部置き換え（in-place changes）が起こること」を意味する。
# アンダースコア（_）がない通常の計算の場合（例えば「add()」）は、計算元のテンソル内部は変更されずに、戻り値として新たなテンソルが取得できる。

### リスト4-4　チートシート「テンソルのインデクシング／スライシング」

In [0]:
# インデクシングやスライシング（NumPyのような添え字を使用可能）
print(x)         # 元は、2行3列のテンソル
x[0, 1]          # 1行2列目（※0スタート）を取得
x[:2, 1:]        # 先頭～2行（＝0行目と1行目）×2列～末尾（＝2列目と3列目）の2行2列が抽出される

### リスト4-5　チートシート「テンソルからPython数値への変換」

In [0]:
# テンソルの1つの要素値を、Pythonの数値に変換
x[0, 1].item()   # 1行2列目（※0スタート）の要素値をPythonの数値で取得

### リスト4-6　チートシート「PyTorchテンソル ←→ NumPy多次元配列値、の変換＆連携」

In [0]:
# PyTorchテンソルを、NumPy多次元配列に変換
b = x.numpy()    # 「numpy()」を呼び出すだけ。以下は注意点（メモリ位置の共有）

# ※PyTorchテンソル側の値を変えると、NumPy多次元配列値「b」も変化する（トラックされる）
print ('PyTorch計算→NumPy反映：')
print(b); x.add_(y); print(b)           # PyTorch側の計算はNumPy側に反映
print ('NumPy計算→PyTorch反映：')
print(x); np.add(b, b, out=b); print(x) # NumPy側の計算はPyTorch側に反映

# -----------------------------------------
# NumPy多次元配列を、PyTorchテンソルに変換
c = np.ones((2, 3), dtype=np.float64) # 2行3列の多次元配列値（1で初期化）を生成
d = torch.from_numpy(c)  # 「torch.from_numpy()」を呼び出すだけ

# ※NumPy多次元配列値を変えると、PyTorchテンソル「b」も変化する（トラックされる）
print ('NumPy計算→PyTorch反映：')
print(d); np.add(c, c, out=c); print(d)  # NumPy側の計算はPyTorch側に反映
print ('PyTorch計算→NumPy反映：')
print(c); d.add_(y); print(c)            # PyTorch側の計算はNumPy側に反映

### リスト4-7　チートシート「テンソルのデータ型の変換」

In [0]:
# データ型の変換（※変換後のテンソルには、NumPyの計算は反映されない）
e = d.float()  # 「torch.float64」から「torch.float32」

### リスト4-8　チートシート「テンソル演算でのGPU利用」

ColabでGPUを有効にするには、メニューバーの［ランタイム］－［ランタイムのタイプを変更］をクリックして切り替えてほしい。

In [0]:
# NVIDIAのGPUである「CUDA」（GPU）デバイス環境が利用可能な場合、
# GPUを使ってテンソルの計算を行うこともできる
if torch.cuda.is_available():              # CUDA（GPU）が利用可能な場合
    print('CUDA（GPU）が利用できる環境')
    print(f'CUDAデバイス数： {torch.cuda.device_count()}')
    print(f'現在のCUDAデバイス番号： {torch.cuda.current_device()}')  # ※0スタート
    print(f'1番目のCUDAデバイス名： {torch.cuda.get_device_name(0)}') # 例「Tesla T4」   

    device = torch.device("cuda")          # デフォルトのCUDAデバイスオブジェクトを取得
    device0 = torch.device("cuda:0")       # 1番目（※0スタート）のCUDAデバイスを取得

    # テンソル計算でのGPUの使い方は主に3つ：
    g = torch.ones(2, 3, device=device)    # （1）テンソル生成時のパラメーター指定
    g = x.to(device)                       # （2）既存テンソルのデバイス変更
    g = x.cuda(device)                     # （3）既存テンソルの「CUDA（GPU）」利用
    f = x.cpu()                            # （3'）既存テンソルの「CPU」利用

    # ※（2）の使い方で、GPUは「.to("cuda")」、CPUは「.to("cpu")」と書いてもよい
    g = x.to("cuda")
    f = x.to("cpu")

    # ※（3）の引数は省略することも可能
    g = x.cuda()

    # 「torch.nn.Module」オブジェクト（model）全体でのGPU／CPUの切り替え
    model.cuda()  # モデルの全パラメーターとバッファーを「CUDA（GPU）」に移行する
    model.cpu()   # モデルの全パラメーターとバッファーを「CPU」に移行する
else:
    print('CUDA（GPU）が利用できない環境')

# 第3回　PyTorchによるディープラーニング実装手順の基本

## ■（5）データセットとデーターローダー（DataLoader）

「[第1回　初めてのニューラルネットワーク実装、まずは準備をしよう ― 仕組み理解×初実装（前編）：TensorFlow 2＋Keras（tf.keras）入門 - ＠IT](https://www.atmarkit.co.jp/ait/articles/1909/19/news026.html)」の記事と同じように、シンプルな座標点データを生成して使う。使い方は、前述の記事を参照してほしい。

なお、座標点データは、「[ニューラルネットワーク Playground - Deep Insider](https://deepinsider.github.io/playground/)」（以下、Playground）と同じ生成仕様となっている。

### リスト5-1　座標点データの生成

In [0]:
# 座標点データを生成するライブラリのインストール
!pip install playground-data

In [0]:
# playground-dataライブラリのplygdataパッケージを「pg」という別名でインポート
import plygdata as pg

# 設定値を定数として定義
PROBLEM_DATA_TYPE = pg.DatasetType.ClassifyCircleData # 問題種別：「分類（Classification）」、データ種別：「円（CircleData）」を選択
TRAINING_DATA_RATIO = 0.5  # データの何％を訓練【Training】用に？ (残りは精度検証【Validation】用) ： 50％
DATA_NOISE = 0.0           # ノイズ： 0％

# 定義済みの定数を引数に指定して、データを生成する
data_list = pg.generate_data(PROBLEM_DATA_TYPE, DATA_NOISE)

# データを「訓練用」と「精度検証用」を指定の比率で分割し、さらにそれぞれを「データ（X）」と「教師ラベル（y）」に分ける
X_train, y_train, X_valid, y_valid = pg.split_data(data_list, training_size=TRAINING_DATA_RATIO)

# データ分割後の各変数の内容例として、それぞれ5件ずつ出力
print('X_train:'); print(X_train[:5])
print('y_train:'); print(y_train[:5])
print('X_valid:'); print(X_valid[:5])
print('y_valid:'); print(y_valid[:5])

### リスト5-2　データセットとデーターローダーの作成

PyTorchにはミニバッチを簡単に扱うための`DataLoader`クラスが用意されている。このクラスを利用するには、既存のデータや教師ラベルといったテンソルを1つの`TensorDataset`にまとめる必要がある。

In [0]:
# データ関連のユーティリティクラスをインポート
from torch.utils.data import TensorDataset, DataLoader
import torch       # ライブラリ「PyTorch」のtorchパッケージをインポート

# 定数（学習方法設計時に必要となるもの）
BATCH_SIZE = 15  # バッチサイズ： 15（Playgroundの選択肢は「1」～「30」）

# NumPy多次元配列からテンソルに変換し、データ型は`float`に変換する
t_X_train = torch.from_numpy(X_train).float()
t_y_train = torch.from_numpy(y_train).float()
t_X_valid = torch.from_numpy(X_valid).float()
t_y_valid = torch.from_numpy(y_valid).float()

# 「データ（X）」と「教師ラベル（y）」を、1つの「データセット（dataset）」にまとめる
dataset_train = TensorDataset(t_X_train, t_y_train)  # 訓練用
dataset_valid = TensorDataset(t_X_valid, t_y_valid)  # 精度検証用

# ミニバッチを扱うための「データローダー（loader）」（訓練用と精度検証用）を作成
loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
loader_valid = DataLoader(dataset_valid, batch_size=BATCH_SIZE)

このコードのポイント：
- バッチサイズは学習時に扱うデータ単位であるが、`DataLoader`が「ミニバッチ」に関係するため、この段階で定義しておく必要がある
- `DataLoader`クラスのコンストラクター引数`shuffle`で、データをシャッフルするか（**True**）しないか（**False**）を指定できる。今回はシャッフルしている

## ■（6）ディープニューラルネットのモデル定義

### リスト6-1　「1」か「-1」に分類するための「出力の離散化」
- 今回のニューラルネットワークでは出力された確率値を、「**1**」か「**-1**」の2クラス分類値に離散化する
  - 具体的には、0.0未満／0.0以上を-1.0／1.0にスケール変換する
- そのための独自関数`discretize`を定義する
- さらに、モデル内で扱いやすいように`torch.nn.Module`化も行っておく（※本稿では使用しない）
- `discretize`関数は後述の「リスト7-3　1回分の「訓練（学習）」と「評価」の処理」で使用する

In [0]:
import torch       # ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn  # 「ニューラルネットワーク」モジュールの別名定義

# 離散化を行う単なる関数
def discretize(proba):
    '''
    実数の確率値を「1」か「-1」の2クラス分類値に離散化する。
    閾値は「0.0以上」か「未満」か。データ型は「torch.float」を想定。
  
    Examples:
        >>> proba = torch.tensor([-0.5, 0.0, 0.5], dtype=torch.float)
        >>> binary = discretize(proba)
    '''
    threshold = torch.Tensor([0.0]) # -1か1かを分ける閾値を作成
    discretized = (proba >= threshold).float() # 閾値未満で0、以上で1に変換
    return discretized * 2 - 1.0 # 2倍して-1.0することで、0／1を-1.0／1.0にスケール変換

# discretize関数をモデルで簡単に使用できるようにするため、
# PyTorchの「torch.nn.Module」を継承したクラスラッパーも作成した
class Discretize(nn.Module):
    '''
    実数の確率値を「1」か「-1」の2クラス分類値に離散化する。
    閾値は「0.0以上」か「未満」か。データ型は「torch.float」を想定。
  
    Examples:
        >>> d = Discretize()
        >>> proba = torch.tensor([-0.5, 0.0, 0.5], dtype=torch.float)
        >>> binary = d(proba)
    '''        
    def __init__(self):
        super().__init__()

    # forward()メソッドは、基本クラス「torch.nn.Module」の__call__メソッドからも呼び出されるため、
    # Discretizeオブジェクトを関数のように使える（例えば上記の「d(proba)」）
    def forward(self, proba):
        return discretize(proba) # 上記の関数を呼び出すだけ

# 関数の利用をテスト
proba = torch.tensor([-0.5, 0.0, 0.5], dtype=torch.float)  # 確率値の例
binary = discretize(proba)  # 2クラス分類（binary classification）値に離散化
binary  # tensor([-1.,  1.,  1.]) …… などと表示される

このコードのポイント：
- PyTrochのニューラルネットワークの基本を解説する内容ではないので、ざっと流して読めば十分（コメントを参考にしてほしい）

### リスト6-2　ディープニューラルネットワークのモデル設計
- 入力の数（`INPUT_FEATURES`）は、$X_1$と$X_2$で**2つ**
- 隠れ層のレイヤー数は、**2つ**
  - 隠れ層にある1つ目のニューロンの数（`LAYER1_NEURONS`）は、**3つ**
  - 隠れ層にある2つ目のニューロンの数（`LAYER2_NEURONS`）は、**3つ**
- 出力層にあるニューロンの数（`OUTPUT_RESULTS`）は、**1つ**

In [0]:
import torch       # ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn  # 「ニューラルネットワーク」モジュールの別名定義

# 定数（モデル定義時に必要となるもの）
INPUT_FEATURES = 2      # 入力（特徴）の数： 2
LAYER1_NEURONS = 3      # ニューロンの数： 3
LAYER2_NEURONS = 3      # ニューロンの数： 3
OUTPUT_RESULTS = 1      # 出力結果の数： 1

# 変数（モデル定義時に必要となるもの）
activation = torch.nn.Tanh()  # 活性化関数（隠れ層用）： tanh関数（変更可能）
act_output = torch.nn.Tanh()  # 活性化関数（出力層用）： tanh関数（固定）

# torch.nn.Moduleによるモデルの定義
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 隠れ層：1つ目のレイヤー（layer）
        self.layer1 = nn.Linear(
            INPUT_FEATURES,                # 入力ユニット数（＝入力層）
            LAYER1_NEURONS)                # 次のレイヤーの出力ユニット数

        # 隠れ層：2つ目のレイヤー（layer）
        self.layer2 = nn.Linear(
            LAYER1_NEURONS,                # 入力ユニット数
            LAYER2_NEURONS)                # 次のレイヤーへの出力ユニット数

        # 出力層
        self.output = nn.Linear(
            LAYER2_NEURONS,                # 入力ユニット数
            OUTPUT_RESULTS)                # 出力結果への出力ユニット数

    def forward(self, x):
        # フィードフォワードを定義
        # 「出力＝活性化関数（第n層（入力））」の形式で記述
        x = activation(self.layer1(x))  # 活性化関数は変数として定義
        x = activation(self.layer2(x))  # 同上
        x = act_output(self.output(x))  # ※活性化関数は「tanh」固定
        return x

# モデル（NeuralNetworkクラス）のインスタンス化
model = NeuralNetwork()
model   # モデルの内容を出力

このコードのポイント：
- 基本的な内容は、前掲の「リスト1-1　ニューロンのモデル設計」と同じ
- 層が1つから、「隠れ層：2＋出力層：1」の3つに拡張されている
- 例えば「LAYER1_NEURONS」に着目すると、1つ目のレイヤーにおける「出力ユニット数」であり、2つ目のレイヤーの「入力ユニット数」でもあるので、全く同じものが指定されている
- フィードフォワード時のデータが変換されていく流れは、`forward`メソッド内に分かりやすく定義されている

## ■（7）学習／最適化（オプティマイザー）

### リスト7-1　オプティマイザー（最適化用オブジェクト）の作成

In [0]:
import torch.optim as optim   # 「最適化」モジュールの別名定義

# 定数（学習方法設計時に必要となるもの）
LEARNING_RATE = 0.03   # 学習率： 0.03
REGULARIZATION = 0.03  # 正則化率： 0.03

# オプティマイザーを作成（パラメーターと学習率も指定）
optimizer = optim.SGD(           # 最適化アルゴリズムに「SGD」を選択
    model.parameters(),          # 最適化で更新対象のパラメーター（重みやバイアス）
    lr=LEARNING_RATE,            # 更新時の学習率
    weight_decay=REGULARIZATION) # L2正則化（※不要な場合は0か省略）

このコードのポイント：
- この例では「SGD」（Stochastic Gradient Descent： 確率的勾配降下法）を選択
- パラメーター（重みやバイアス）と、学習率、正則化率を引数に指定
- 正則化（regularization）は「L2」に相当する。あまり使わない「L1」は基本機能に含まれていない
- `torch.optim.SGD`を含めて以下が使用可能
  - Adadelta
  - Adagrad
  - Adam（有名）
  - AdamW
  - SparseAdam
  - Adamax
  - ASGD
  - LBFGS
  - RMSprop
  - Rprop
  - SGD（確率的勾配降下法）

### リスト7-2　損失関数の定義
定義した損失関数は次のリスト7-3で利用する。

In [0]:
# 変数（学習方法設計時に必要となるもの）
criterion = nn.MSELoss()  # 損失関数：平均二乗誤差

このコードのポイント：
- `criterion`は慣例の変数名。誤差からの損失を測る「基準（criterion）」を意味する
- `nn.MSELoss`も含めて以外が使用可能
  - L1Loss（MAE：Mean Absolute Error、平均絶対誤差）
  - MSELoss（MSE：Mean Squared Error、平均二乗誤差）
  - CrossEntropyLoss（交差エントロピー誤差： クラス分類）
  - CTCLoss
  - NLLLoss
  - PoissonNLLLoss
  - KLDivLoss
  - BCELoss
  - BCEWithLogitsLoss
  - MarginRankingLoss
  - HingeEmbeddingLoss
  - MultiLabelMarginLoss
  - SmoothL1Loss
  - SoftMarginLoss
  - MultiLabelSoftMarginLoss
  - CosineEmbeddingLoss
  - MultiMarginLoss
  - TripletMarginLoss


### リスト7-3　1回分の「訓練（学習）」と「評価」の処理
`train_step`関数に訓練の内容を、`valid_step`関数に評価の内容を記述する。

In [0]:
def train_step(train_X, train_y):
    # 訓練モードに設定
    model.train()

    # フィードフォワードで出力結果を取得
    #train_X                # 入力データ
    pred_y = model(train_X) # 出力結果
    #train_y                # 正解ラベル

    # 出力結果と正解ラベルから損失を計算し、勾配を求める
    optimizer.zero_grad()   # 勾配を0で初期化（※累積してしまうため要注意）
    loss = criterion(pred_y, train_y)     # 誤差（出力結果と正解ラベルの差）から損失を取得
    loss.backward()   # 逆伝播の処理として勾配を計算（自動微分）

    # 勾配を使ってパラメーター（重みとバイアス）を更新
    optimizer.step()  # 指定されたデータ分の最適化を実施

    # 正解率を算出
    with torch.no_grad(): # 勾配は計算しないモードにする
        discr_y = discretize(pred_y)         # 確率値から「-1」／「1」に変換
        acc = (discr_y == train_y).sum()     # 正解数を計算する

    # 損失と正解数をタプルで返す
    return (loss.item(), acc.item())  # ※item()=Pythonの数値

def valid_step(valid_X, valid_y):
    # 評価モードに設定（※dropoutなどの挙動が評価用になる）
    model.eval()
    
    # フィードフォワードで出力結果を取得
    #valid_X                # 入力データ
    pred_y = model(valid_X) # 出力結果
    #valid_y                # 正解ラベル

    # 出力結果と正解ラベルから損失を計算
    loss = criterion(pred_y, valid_y)     # 誤差（出力結果と正解ラベルの差）から損失を取得
    # ※評価時は勾配を計算しない

    # 正解率を算出
    with torch.no_grad(): # 勾配は計算しないモードにする
        discr_y = discretize(pred_y)     # 確率値から「-1」／「1」に変換
        acc = (discr_y == valid_y).sum() # 正解数を合計する

    # 損失と正解数をタプルで返す
    return (loss.item(), acc.item())  # ※item()=Pythonの数値

このコードのポイント：
- `model.eval()`メソッドを呼び出すと、評価（推論）モードとなり、（今回は使っていないが）BatchNormやDropoutなどの挙動が評価用になる。通常は、訓練モード（`model.train()`メソッド）になっている
- `train_step`関数の処理内容： 「訓練モードの設定」「フィードフォワードで出力結果の取得」「出力結果と正解ラベルから損失および勾配の計算」「勾配を使ってパラメーター（重みとバイアス）の更新」「正解率の算出」
- `valid_step`関数の処理内容： 「評価モードの設定」「フィードフォワードで出力結果の取得」「出力結果と正解ラベルから損失の計算」「正解率の算出」
- フィードフォワードは、前掲の「リスト2-1　フィードフォワードの実行と結果確認」で説明済み
- `with torch.no_grad():`の配下のテンソル計算のコードには自動微分用の勾配が生成されなくなり、メモリ使用量軽減やスピードアップなどの効果がある
- `discretize`関数は前掲の『リスト6-1　「1」か「-1」に分類するための「出力の離散化」』で説明した

### リスト7-4　「訓練」と「評価」をバッチサイズ単位でエポック回繰り返す
`train_step`関数で訓練を、`valid_step`関数で評価を実行する。早期終了は基本機能ではないので今回は説明しない。

In [0]:
# パラメーター（重みやバイアス）の初期化を行う関数の定義
def init_parameters(layer):
    if type(layer) == nn.Linear:
        nn.init.xavier_uniform_(layer.weight) # 重みを「一様分布のランダム値」に初期化
        layer.bias.data.fill_(0.0)            # バイアスを「0」に初期化

# 学習の前にパラメーター（重みやバイアス）を初期化する
model.apply(init_parameters)

# 定数（学習／評価時に必要となるもの）
EPOCHS = 100             # エポック数： 100

# 変数（学習／評価時に必要となるもの）
avg_loss = 0.0           # 「訓練」用の平均「損失値」
avg_acc = 0.0            # 「訓練」用の平均「正解率」
avg_val_loss = 0.0       # 「評価」用の平均「損失値」
avg_val_acc = 0.0        # 「評価」用の平均「正解率」

# 損失の履歴を保存するための変数
train_history = []
valid_history = []

for epoch in range(EPOCHS):
    # forループ内で使う変数と、エポックごとの値リセット
    total_loss = 0.0     # 「訓練」時における累計「損失値」
    total_acc = 0.0      # 「訓練」時における累計「正解数」
    total_val_loss = 0.0 # 「評価」時における累計「損失値」
    total_val_acc = 0.0  # 「評価」時における累計「正解数」
    total_train = 0      # 「訓練」時における累計「データ数」
    total_valid = 0      # 「評価」時における累計「データ数」

    for train_X, train_y in loader_train:
        # 【重要】1ミニバッチ分の「訓練」を実行
        loss, acc = train_step(train_X, train_y)

        # 取得した損失値と正解率を累計値側に足していく
        total_loss += loss          # 訓練用の累計損失値
        total_acc += acc            # 訓練用の累計正解数
        total_train += len(train_y) # 訓練データの累計数
            
    for valid_X, valid_y in loader_valid:
        # 【重要】1ミニバッチ分の「評価（精度検証）」を実行
        val_loss, val_acc = valid_step(valid_X, valid_y)

        # 取得した損失値と正解率を累計値側に足していく
        total_val_loss += val_loss  # 評価用の累計損失値
        total_val_acc += val_acc    # 評価用の累計正解数
        total_valid += len(valid_y) # 訓練データの累計数

    # ミニバッチ単位で累計してきた損失値や正解率の平均を取る
    n = epoch + 1                             # 処理済みのエポック数
    avg_loss = total_loss / n                 # 訓練用の平均損失値
    avg_acc = total_acc / total_train         # 訓練用の平均正解率
    avg_val_loss = total_val_loss / n         # 訓練用の平均損失値
    avg_val_acc = total_val_acc / total_valid # 訓練用の平均正解率

    # グラフ描画のために損失の履歴を保存する
    train_history.append(avg_loss)
    valid_history.append(avg_val_loss)

    # 損失や正解率などの情報を表示
    print(f'[Epoch {epoch+1:3d}/{EPOCHS:3d}]' \
          f' loss: {avg_loss:.5f}, acc: {avg_acc:.5f}' \
          f' val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f}')

print('Finished Training')
print(model.state_dict())  # 学習後のパラメーターの情報を表示

このコードのポイント：
- 重要なのは、「リスト7-3　1回分の「訓練（学習）」と「評価」の処理」で定義した`train_step`関数と`valid_step`関数の呼び出しだけである
- 1つ目のforループでエポックを回している
  - 2つ目のforループでバッチ単位分のデータを処理に渡している（ミニバッチ処理）
- たくさんの変数があって長いが、どれも表示用の損失や正解率を計算するための処理であり、ニューラルネットワークの本質ではない

## ■（8）評価／精度検証

### リスト8-1　損失値の推移グラフ描画

In [0]:
import matplotlib.pyplot as plt

# 学習結果（損失）のグラフを描画
epochs = len(train_history)
plt.plot(range(epochs), train_history, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_history, marker='.', label='loss (validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

このコードのポイント：
- `train_history`と`valid_history`は、前述の「リスト7-4　「訓練」と「評価」をバッチサイズ単位でエポック回繰り返す」で記録しておいた損失の履歴である
- プロットする方法はいたって普通なので、特に本稿では説明しない

# お疲れさまでした。第1回～第3回は修了です。

- 「第1回　難しくない！ PyTorchでニューラルネットワークの基本」
- 「第2回　PyTorchのテンソル＆データ型のチートシート」
- 「第3回　PyTorchによるディープラーニング実装手順の基本」
